In [27]:
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [2]:
# Initializing driver
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [4]:
# First website to scrape
nasa_url = 'https://mars.nasa.gov/news/'
browser.visit(nasa_url)
html = browser.html
soup = BeautifulSoup(html, "html.parser")
# soup.find_all('ul', class_='item_list')

In [5]:
# Find title and excerpt with beautiful soup
html = browser.html
soup = BeautifulSoup(html, "html.parser")
news_slide = soup.find('div', class_='list_text')
title = news_slide.find('div', class_='content_title').find('a').text
paragraph = news_slide.find('div', class_="article_teaser_body").text
paragraph

"As part of NASA's 'Send Your Name to Mars' campaign, they've been stenciled onto three microchips along with essays from NASA's 'Name the Rover' contest. Next stop: Mars."

In [6]:
# Second website to scrape
img_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(img_url)
# Navagate to details page for full res link
browser.click_link_by_partial_text('FULL IMAGE')
browser.click_link_by_partial_text('more info')
# parse into beautiful soup
html = browser.html
soup = BeautifulSoup(html, "lxml")
# find download link, [1] because the jpeg link is the second one
img_link = soup.find_all(class_ = "download_tiff")[1].p.a['href']
img_link

'//photojournal.jpl.nasa.gov/jpeg/PIA16605.jpg'

In [28]:
# Third scrape
twitter_url = "https://twitter.com/marswxreport?lang=en"
html = requests.get(twitter_url)
soup = BeautifulSoup(html.text, 'html.parser')
# Using some code from office hours below
timeline = soup.select('#timeline li.stream-item')
for tweet in timeline:
    tweet_text = tweet.select('p.tweet-text')[0].get_text()
    # first tweet that contain weather strings, store weather and break
    if tweet_text.find('ºC')!= -1 and tweet_text.lower().find('sol') != -1:
        weather = tweet_text
        break
weather.split('hPapic')[0]

'InSight sol 457 (2020-03-10) low -95.7ºC (-140.3ºF) high -9.1ºC (15.6ºF)\nwinds from the SSE at 6.5 m/s (14.5 mph) gusting to 21.0 m/s (46.9 mph)\npressure at 6.30 '

In [7]:
# Read html into panda data frame and then covert back only the table
facts_url = 'https://space-facts.com/mars/'
facts_tables = pd.read_html(facts_url)
facts_tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:    -153 to 20 °C      -88 to 58°C,
           

In [8]:
table_string = facts_tables[0].to_html()
table_string

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>0</th>\n      <th>1</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:</td>\n      <td>2nd millennium BC</td>\n    <

In [9]:
# visit website with driver
hemi_url="https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemi_url)
hemi_urls = []
img_links = browser.find_link_by_partial_text('Enhanced')
for i in range(len(img_links)):
    # for each img link needs to grap link again after each visit
    browser.find_link_by_partial_text('Enhanced')[i].click()
    # parse into beautiful soup
    html = browser.html
    soup = BeautifulSoup(html, "lxml")
    # add dictionaries to list
    # found titles and links with beautiful soup
    # Grapping jpeg instead of tif
    hemi_urls.append({'title': soup.find('title').text.split('Enhanced')[0][0:-1], 'img_url': soup.find('div', class_='downloads').ul.find_all('li')[0].a['href']})
    # go back to search page
    browser.back()
hemi_urls

[{'title': 'Cerberus Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif'},
 {'title': 'Schiaparelli Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif'},
 {'title': 'Syrtis Major Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif'},
 {'title': 'Valles Marineris Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif'}]